In [1]:
import Base.*
import Base.convert
import Base.promote_rule

In [2]:
struct Gn{N}
    x
    function Gn{N}(x::Int64) where N
        x = x % N
        if gcd(x, N) != 1
            throw(DomainError())
        else
            new(x)
        end
    end
end


In [27]:
println("Gn{7}(4): ", Gn{7}(4))
println("Gn{7}(9): ", Gn{7}(9))
Gn{7}(1000)

Gn{7}(4): Gn{7}(4)
Gn{7}(9): Gn{7}(2)


Gn{7}(6)

In [4]:
Gn{6}(4)

LoadError: DomainError:

In [5]:
function group_size(::Type{Gn{N}}) where N
    count = 0
    for i = 1:(N - 1)
        if(gcd(i, N) == 1)
            count += 1
        end
    end
    count
end

group_size (generic function with 1 method)

In [6]:
println("Gn{7}: ", group_size(Gn{7}))
println("Gn{8}: ", group_size(Gn{8}))

Gn{7}: 6
Gn{8}: 4


In [7]:
function *(x::Gn{N}, y::Gn{N}) where N
    Gn{N}(x.x * y.x)
end

function *(x::Gn{N}, y::T) where N where T <: Integer
    Gn{N}(x.x * y) 
end
function *(x::T, y::Gn{N}) where N where T <: Integer
    Gn{N}(x * y.x)
end


* (generic function with 185 methods)

In [28]:
println("Gn{7}(3) * 4: ", Gn{7}(3) * 4)
println("3 * Gn{7}(4): ", 3 * Gn{7}(4))
println("Gn{7}(3) * Gn{7}(4): ", Gn{7}(3) * Gn{7}(4))

Gn{7}(3) * 4: Gn{7}(5)
3 * Gn{7}(4): Gn{7}(5)
Gn{7}(3) * Gn{7}(4): Gn{7}(5)


In [9]:
convert(::Type{Int64}, x::Gn{N}) where N = x.x
convert(::Type{Gn{N}}, x::Int64) where N = Gn{N}(x)

convert (generic function with 723 methods)

In [29]:
function test_convert()
    x::Gn{3} = 5
    y::Int64 = Gn{7}(11)
    x, y
end
test_convert()

(Gn{3}(2), 4)

In [11]:
promote_rule(::Type{Gn{N}}, ::Type{T}) where N where T <: Integer = Int64

promote_rule (generic function with 124 methods)

In [12]:
promote_type(Int8, Gn{3})

Int64

In [13]:
function powmod(a::Gn{N}, x::T) where N where T <: Integer
    result = 1
    for _ = 1:x
        result *= a
    end
    result
end


powmod (generic function with 1 method)

In [14]:
powmod(Gn{8}(3), 3)

Gn{8}(3)

In [15]:
function period(a::Gn{N}) where N
    r = 1
    elems = group_size(Gn{N})
    for r = 1:elems
        if elems % r != 0
            continue
        end
        if powmod(a, r) == Gn{N}(1)
            return r
        end
    end
    throw(ErrorException("Period not found"))
end

period (generic function with 1 method)

In [16]:
period(Gn{3}(2))

2

Wikipedia says:
```pascal
function inverse(a, n)
    t := 0;     newt := 1;    
    r := n;     newr := a;    
    while newr ≠ 0
        quotient := r div newr
        (t, newt) := (newt, t - quotient * newt) 
        (r, newr) := (newr, r - quotient * newr)
    if r > 1 then return "a is not invertible"
    if t < 0 then t := t + n
    return t
```


In [17]:
function inverse(a::Gn{N}) where N
    t::Int64 = 0
    r::Int64 = N
    newt::Int64 = 1
    newr::Int64 = a
    while newr !=0 
        quotient = div(r, newr)
        (t, newt) = (newt, t - quotient * newt)
        (r, newr) = (newr, r - quotient * newr)
    end
    if r > 1
        throw(DomainError())
    end
    if t < 0
        t += N
    end
    Gn{N}(t)
end

inverse (generic function with 1 method)

In [24]:
inverse(Gn{7}(5))

Gn{7}(3)

## RSA

In [20]:
N = 55
c = 17
b = 4


4

In [25]:
function decrypt(keyN, keyC, msg)
    p = period(Gn{keyN}(msg))
    privkey = inverse(Gn{keyN}(keyC)) 
    plain = powmod(Gn{keyN}(msg), privkey.x)
    
    assert(Int64(powmod(Gn{keyN}(plain), keyC)) == msg)
    
    Int64(plain)
end


decrypt (generic function with 1 method)

In [26]:
decrypt(N, c, b)

9